In [107]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

ratings = pd.read_csv('./archive (2)/ratings.csv')
ratings = ratings.drop(columns=['timestamp'])

print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]


In [109]:
# Đếm số lượng item mỗi user đánh gias
user_counts = ratings['userId'].value_counts()
# Loại bỏ User có ít đánh giá
user_counts = user_counts[user_counts >= 50]

# Đếm số lượng user đánh giá mỗi item
movie_counts = ratings['movieId'].value_counts()
# Đếm số lượng đánh giá của item
movie_counts = movie_counts[movie_counts >= 50]

#ratings sau khi lọc dữ liệu
ratings = ratings[ratings['movieId'].isin(movie_counts.index)]
ratings = ratings[ratings['userId'].isin(user_counts.index)]
ratings



,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100657,610,106782,4.5
100672,610,109374,4.5
100673,610,109487,3.5
100701,610,112852,4.5


In [111]:
test_list = []

# Group theo userId
for user_id, group in ratings.groupby('userId'):
    group = group.sample(frac=1, random_state=42)  # shuffle dữ liệu của user
    test_size = max(1, int(len(group) * 0.2))  
    test_list.append(group.iloc[:test_size])

test_df = pd.concat(test_list)
train_df = ratings.drop(test_df.index)

train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [113]:
# Đọc dữ liệu từ file train.csv
train_data = pd.read_csv('train.csv')
# Tạo utility matrix
utility_matrix = train_data.pivot(index='userId', columns='movieId', values='rating')
# Hiển thị ma trận
utility_matrix

movieId,1,2,3,6,7,10,11,16,17,19,...,91500,91529,96079,99114,106782,109374,109487,112852,116797,122904
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,5.0,4.0,4.0,NaN,4.0,4.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,1.0,0.5,0.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,2.5,NaN,NaN,NaN,2.5,NaN,2.5,NaN,4.0,2.0,...,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
user_mean = utility_matrix.mean(axis=1)
print(user_mean)

userId
1      4.319149
4      3.514706
6      3.880000
7      3.635135
10     3.111111
         ...   
605    3.533898
606    3.748691
607    3.787500
608    3.372984
610    4.281250
Length: 385, dtype: float64


In [117]:
normalized_utility_matrix_NaN = utility_matrix.sub(user_mean, axis=0)
print(normalized_utility_matrix_NaN)

movieId    1         2         3         6         7       10        11      \
userId                                                                        
1             NaN       NaN -0.319149 -0.319149       NaN     NaN       NaN   
4             NaN       NaN       NaN       NaN       NaN     NaN       NaN   
6             NaN       NaN  1.120000  0.120000  0.120000     NaN  0.120000   
7             NaN       NaN       NaN       NaN       NaN     NaN       NaN   
10            NaN       NaN       NaN       NaN       NaN     NaN       NaN   
...           ...       ...       ...       ...       ...     ...       ...   
605           NaN -0.033898       NaN       NaN       NaN     NaN       NaN   
606     -1.248691       NaN       NaN       NaN -1.248691     NaN -1.248691   
607           NaN       NaN       NaN       NaN       NaN     NaN -0.787500   
608     -0.872984 -1.372984 -1.372984       NaN       NaN     NaN       NaN   
610      0.718750       NaN       NaN  0.718750     

In [119]:
normalized_utility_matrix = normalized_utility_matrix_NaN.fillna(0)
print(normalized_utility_matrix)

movieId    1         2         3         6         7       10        11      \
userId                                                                        
1        0.000000  0.000000 -0.319149 -0.319149  0.000000     0.0  0.000000   
4        0.000000  0.000000  0.000000  0.000000  0.000000     0.0  0.000000   
6        0.000000  0.000000  1.120000  0.120000  0.120000     0.0  0.120000   
7        0.000000  0.000000  0.000000  0.000000  0.000000     0.0  0.000000   
10       0.000000  0.000000  0.000000  0.000000  0.000000     0.0  0.000000   
...           ...       ...       ...       ...       ...     ...       ...   
605      0.000000 -0.033898  0.000000  0.000000  0.000000     0.0  0.000000   
606     -1.248691  0.000000  0.000000  0.000000 -1.248691     0.0 -1.248691   
607      0.000000  0.000000  0.000000  0.000000  0.000000     0.0 -0.787500   
608     -0.872984 -1.372984 -1.372984  0.000000  0.000000     0.0  0.000000   
610      0.718750  0.000000  0.000000  0.718750  0.0

In [121]:
#hàm tính độ tương đồng cosine
def cosine_similarity_matrix(normalized_utility_matrix):
    matrix = normalized_utility_matrix
    labels = matrix.index  # userId
    similarity = cosine_similarity(matrix)
    # Đưa về DataFrame với chỉ số và cột là movieId hoặc userId
    similarity_df = pd.DataFrame(similarity, index=labels, columns=labels)
    return similarity_df


In [123]:
user_similarity_matrix = cosine_similarity_matrix(normalized_utility_matrix)
print("\nMa trận tương đồng giữa các người dùng (user-based):")
print(user_similarity_matrix)


Ma trận tương đồng giữa các người dùng (user-based):
userId       1         4         6         7         10        11        15   \
userId                                                                         
1       1.000000  0.061542 -0.107204 -0.064374  0.021809 -0.044157  0.039639   
4       0.061542  1.000000 -0.025005  0.039320  0.153384  0.181386  0.034075   
6      -0.107204 -0.025005  1.000000 -0.025479  0.020735  0.122204  0.053218   
7      -0.064374  0.039320 -0.025479  1.000000 -0.043030  0.014411 -0.020505   
10      0.021809  0.153384  0.020735 -0.043030  1.000000  0.017798  0.012572   
...          ...       ...       ...       ...       ...       ...       ...   
605    -0.035237  0.112672 -0.026137  0.059802  0.088070  0.026384  0.000668   
606     0.014362  0.011801 -0.051915  0.031477 -0.102789 -0.006273  0.040017   
607    -0.006194  0.004567  0.121867  0.037971 -0.000310  0.051698 -0.002285   
608     0.039710 -0.086335 -0.026086 -0.040407 -0.100618 -0.006213

In [125]:
def predict_normalized_rating(user_id, item_id, normalized_matrix, similarity_matrix, k):
    ratings_vector = normalized_matrix[:, item_id]  # rating cho item bởi các user
    rated_entities = np.where(ratings_vector != 0)[0]
    if rated_entities.size == 0:
        return 0
    sims = similarity_matrix[user_id, rated_entities]
    top_k_idx = np.argsort(-np.abs(sims))[:k]
    top_k_sims = sims[top_k_idx]
    top_k_ratings = ratings_vector[rated_entities[top_k_idx]]

   
    numerator = np.dot(top_k_sims, top_k_ratings)
    denominator = np.sum(np.abs(top_k_sims))
    return numerator / denominator if denominator != 0 else 0


In [127]:

normalized_matrix = normalized_utility_matrix.to_numpy()
similarity_matrix_np = user_similarity_matrix.to_numpy()
row_means_np = user_mean.to_numpy()  # hoặc user_mean nếu item-based

completed_matrix = normalized_matrix.copy().astype(np.float32)
num_users, num_items = completed_matrix.shape



# Dự đoán các giá trị còn thiếu
for user_id in range(num_users):
    for item_id in range(num_items):
        if completed_matrix[user_id, item_id] == 0:
            completed_matrix[user_id, item_id] = predict_normalized_rating(
                user_id=user_id,
                item_id=item_id,
                normalized_matrix=normalized_matrix,
                similarity_matrix=similarity_matrix_np,
                k=50)

for user_id in range(num_users):
    completed_matrix[user_id, :] += row_means_np[user_id]

# Output kết quả
predicted_matrix = completed_matrix
print("Utility matrix đã hoàn thiện:")
print(predicted_matrix)


Utility matrix đã hoàn thiện:
[[4.480133  3.8828852 4.        ... 4.516401  4.4831696 4.404417 ]
 [3.7135026 3.4480348 3.119832  ... 3.5620878 3.8212936 3.4779398]
 [3.9840488 4.060448  5.        ... 4.189796  3.8490412 4.01308  ]
 ...
 [3.7794302 3.637161  4.0418286 ... 3.8375385 4.042557  3.454832 ]
 [2.5       2.        2.        ... 3.42876   3.1580412 3.1979353]
 [5.        3.8197637 3.9331698 ... 4.5       4.089192  3.       ]]


In [128]:
# Giữ lại index và columns gốc của ma trận normalized_utility_matrix (giả sử là DataFrame)
index = normalized_utility_matrix.index
columns = normalized_utility_matrix.columns

# Chuyển numpy array về DataFrame
predicted_df = pd.DataFrame(predicted_matrix, index=index, columns=columns)

print("Utility matrix đã hoàn thiện dưới dạng DataFrame:")
predicted_df

Utility matrix đã hoàn thiện dưới dạng DataFrame:


movieId,1,2,3,6,7,10,11,16,17,19,...,91500,91529,96079,99114,106782,109374,109487,112852,116797,122904
userId,,,,,,,,,,,,,,,,,,,,,
1,4.480133,3.882885,4.000000,4.000000,4.060598,4.021080,4.070447,4.518784,4.241352,4.065014,...,3.826652,4.353799,4.365094,4.407174,4.471583,4.396770,4.382883,4.516401,4.483170,4.404417
4,3.713503,3.448035,3.119832,3.415637,3.602625,3.254765,3.600038,3.392939,3.594708,3.258492,...,3.749769,3.507195,3.534018,3.441046,3.089813,3.227837,3.414060,3.562088,3.821294,3.477940
6,3.984049,4.060448,5.000000,4.000000,4.000000,3.868613,4.000000,4.000000,4.000000,2.000000,...,3.963736,3.941041,3.835480,3.751827,4.024545,3.993233,3.633867,4.189796,3.849041,4.013080
7,3.943062,3.392447,3.302025,3.759824,3.125108,3.463263,3.522674,3.853124,3.713778,2.862716,...,3.278649,3.868206,3.724367,3.675132,3.485189,3.793818,3.907904,3.847239,3.948064,3.814798
10,3.066254,3.306329,3.069005,2.907068,3.380522,3.317028,3.311315,2.836444,3.363779,3.359329,...,3.413867,5.000000,2.962691,3.007175,1.000000,0.500000,0.500000,3.506543,3.231233,3.145615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,3.719568,3.500000,2.942768,3.583958,3.380380,3.564842,3.585079,3.564595,3.591462,3.280693,...,3.263467,3.791468,3.526079,3.462518,3.016719,3.521936,3.538270,3.821603,3.923257,3.550868
606,2.500000,3.326086,3.412204,3.852966,2.500000,3.387399,2.500000,4.121753,4.000000,2.000000,...,4.500000,3.745021,3.795301,3.911280,3.981574,4.070885,3.823216,3.784079,3.722305,3.611745
607,3.779430,3.637161,4.041829,3.932148,3.489877,3.707818,3.000000,3.981294,3.837719,3.195908,...,3.919072,3.911774,3.841654,4.026619,3.720713,4.070694,3.842457,3.837538,4.042557,3.454832


In [131]:
from sklearn.metrics import mean_squared_error

# Đọc file train.csv để xây ánh xạ userId và movieId -> chỉ số ma trận
train_df = pd.read_csv("train.csv")

# Tạo ánh xạ từ userId và movieId về chỉ số (index)
userId_mapping = {id_: idx for idx, id_ in enumerate(train_df['userId'].unique())}
movieId_mapping = {id_: idx for idx, id_ in enumerate(train_df['movieId'].unique())}

# Đọc file test.csv
test_df = pd.read_csv("test.csv")

# Danh sách chứa rating thực và rating dự đoán
y_true = []
y_pred = []

# Duyệt từng dòng test.csv và so sánh với predicted_matrix
for _, row in test_df.iterrows():
    user_id_orig = int(row["userId"])
    movie_id_orig = int(row["movieId"])
    actual_rating = float(row["rating"])

    # Kiểm tra xem user/movie có nằm trong ma trận không
    if user_id_orig in userId_mapping and movie_id_orig in movieId_mapping:
        user_idx = userId_mapping[user_id_orig]
        item_idx = movieId_mapping[movie_id_orig]

        # Lấy giá trị dự đoán từ ma trận
        predicted_rating = predicted_matrix[user_idx][item_idx]

        y_true.append(actual_rating)
        y_pred.append(predicted_rating)

# Tính RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print("RMSE =", rmse)



RMSE = 1.086799734419859


In [133]:

# Đọc dữ liệu test
test_df = pd.read_csv("test.csv")

# Danh sách chứa rating thực và rating dự đoán
y_true = []
y_pred = []

# Duyệt qua từng dòng trong test_df để lấy giá trị thực tế và giá trị dự đoán
for _, row in test_df.iterrows():
    user_id_orig = int(row["userId"])
    movie_id_orig = int(row["movieId"])
    actual_rating = float(row["rating"])

    # Kiểm tra xem user/movie có nằm trong ma trận không
    if user_id_orig in userId_mapping and movie_id_orig in movieId_mapping:
        user_idx = userId_mapping[user_id_orig]
        item_idx = movieId_mapping[movie_id_orig]

        # Lấy giá trị dự đoán từ ma trận
        predicted_rating = predicted_matrix[user_idx][item_idx]

        y_true.append(actual_rating)
        y_pred.append(predicted_rating)

# Tính MAE
mae = np.mean(np.abs(np.array(y_true) - np.array(y_pred)))
print("MAE =", mae)


MAE = 0.8215724753670676


In [135]:
# Đọc dữ liệu test
test_df = pd.read_csv("test.csv")

# Danh sách chứa rating thực và rating dự đoán
y_true = []
y_pred = []

# Duyệt qua từng dòng trong test_df để lấy giá trị thực tế và giá trị dự đoán
for _, row in test_df.iterrows():
    user_id_orig = int(row["userId"])
    movie_id_orig = int(row["movieId"])
    actual_rating = float(row["rating"])

    # Kiểm tra xem user/movie có nằm trong ma trận không
    if user_id_orig in userId_mapping and movie_id_orig in movieId_mapping:
        user_idx = userId_mapping[user_id_orig]
        item_idx = movieId_mapping[movie_id_orig]

        # Lấy giá trị dự đoán từ ma trận
        predicted_rating = predicted_matrix[user_idx][item_idx]

        y_true.append(actual_rating)
        y_pred.append(predicted_rating)

# Tính MSE
mse = np.mean((np.array(y_true) - np.array(y_pred))**2)
print("MSE =", mse)


MSE = 1.1811336627350761


In [137]:
import pandas as pd
import numpy as np

# Đọc dữ liệu
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Danh sách user và movie duy nhất
unique_user_ids = sorted(train_df['userId'].unique())
unique_movie_ids = sorted(train_df['movieId'].unique())

# Mapping userId/movieId sang index
userId_to_index = {uid: idx for idx, uid in enumerate(unique_user_ids)}
movieId_to_index = {mid: idx for idx, mid in enumerate(unique_movie_ids)}

def get_top_k_predictions_for_user(user_idx, seen_indices, K):
    user_ratings = predicted_matrix[user_idx, :].copy()

    # Loại bỏ phim đã xem (đã có trong tập train) bằng cách gán -inf
    #user_ratings[seen_indices] = -np.inf

    top_k_movie_indices = np.argsort(user_ratings)[::-1][:K]
    return top_k_movie_indices

def evaluate_top_k(predicted_matrix, train_df, test_df,
                   userId_to_index, movieId_to_index,
                   unique_user_ids, max_k, threshold):
    for K in range(10, max_k + 1):
        precisions = []
        recalls = []

        for user_id in unique_user_ids:
            if user_id not in userId_to_index:
                continue

            user_idx = userId_to_index[user_id]

            # === Danh sách phim đã xem trong tập train ===
            seen_movies = train_df[train_df['userId'] == user_id]['movieId'].values
            seen_indices = [movieId_to_index[mid] for mid in seen_movies if mid in movieId_to_index]

            # === Danh sách phim user thích trong test (rating >= threshold) ===
            user_test_ratings = test_df[
                (test_df['userId'] == user_id) &
                (test_df['movieId'].isin(movieId_to_index))
            ]
            liked_movies = user_test_ratings[
                user_test_ratings['rating'] >= threshold
            ]['movieId'].values

            if len(liked_movies) == 0:
                continue

            liked_movie_indices = [
                movieId_to_index[mid] for mid in liked_movies if mid in movieId_to_index
            ]

            # === Lấy top-K phim dự đoán (loại bỏ phim đã xem) ===
            top_k_indices = get_top_k_predictions_for_user(user_idx, seen_indices, K)

            hits = len(set(top_k_indices).intersection(set(liked_movie_indices)))

            precision = hits / K
            recall = hits / len(liked_movie_indices)

            precisions.append(precision)
            recalls.append(recall)

        # Tính trung bình các chỉ số
        mean_precision = np.mean(precisions) if precisions else 0
        mean_recall = np.mean(recalls) if recalls else 0

        print(f"K={K:2d} | Mean Precision@{K}: {mean_precision:.4f} | Mean Recall@{K}: {mean_recall:.4f}")

evaluate_top_k(
    predicted_matrix,
    train_df,
    test_df,
    userId_to_index,
    movieId_to_index,
    unique_user_ids,
    max_k=30,
    threshold=3.5
)


K=10 | Mean Precision@10: 0.0120 | Mean Recall@10: 0.0175
K=11 | Mean Precision@11: 0.0116 | Mean Recall@11: 0.0186
K=12 | Mean Precision@12: 0.0122 | Mean Recall@12: 0.0207
K=13 | Mean Precision@13: 0.0128 | Mean Recall@13: 0.0234
K=14 | Mean Precision@14: 0.0140 | Mean Recall@14: 0.0265
K=15 | Mean Precision@15: 0.0153 | Mean Recall@15: 0.0316
K=16 | Mean Precision@16: 0.0163 | Mean Recall@16: 0.0351
K=17 | Mean Precision@17: 0.0169 | Mean Recall@17: 0.0385
K=18 | Mean Precision@18: 0.0179 | Mean Recall@18: 0.0420
K=19 | Mean Precision@19: 0.0185 | Mean Recall@19: 0.0455
K=20 | Mean Precision@20: 0.0193 | Mean Recall@20: 0.0487
K=21 | Mean Precision@21: 0.0198 | Mean Recall@21: 0.0515
K=22 | Mean Precision@22: 0.0206 | Mean Recall@22: 0.0549
K=23 | Mean Precision@23: 0.0212 | Mean Recall@23: 0.0588
K=24 | Mean Precision@24: 0.0217 | Mean Recall@24: 0.0645
K=25 | Mean Precision@25: 0.0226 | Mean Recall@25: 0.0677
K=26 | Mean Precision@26: 0.0238 | Mean Recall@26: 0.0733
K=27 | Mean Pr